In [2]:
import pandas
from libs import open_file, replace_symbol, sql_tokenizer,keys
import libs

In [3]:
xss = pandas.read_csv("csv_files/xss-v2.csv")
sql = pandas.read_csv("csv_files/sql-v2.csv")
safe = open_file("datasets/safe.txt","safe")

In [7]:
xss["label"] = "xss"
sql["label"] = "sql"
safe["label"] = "safe"

In [8]:
xss=xss.rename(columns={'Sentence': 'text'})
sql=sql.rename(columns={'Query': 'text'})
safe=safe.rename(columns={"text":"text"})

In [9]:
xss['text'] = xss['text'].apply(sql_tokenizer)
sql['text'] = sql['text'].apply(sql_tokenizer)
safe['text'] = safe['text'].apply(sql_tokenizer)

In [9]:
xss.head()

,Unnamed: 0,text,Label,label
0,0,LT li GT LT A href EQ SLSH wiki SLSH FILE CLN ...,0,xss
1,1,LT tt onmouseover EQ alert LPRN INT RPRN GT te...,1,xss
2,2,LT SLSH span GT LT span CLASS EQ reference MIN...,0,xss
3,3,LT SLSH span GT LT span CLASS EQ reference MIN...,0,xss
4,4,LT SLSH span GT DOT LT A href EQ SLSH wiki SLS...,0,xss


In [10]:
print(len(xss))
print(len(sql))
print(len(safe))

13686
30919
25000


In [11]:
max_num = 5000
xss_n=xss.head(max_num)
sql_n = sql.head(max_num)
safe_n = safe.head(max_num)

In [12]:
print(len(xss_n))
print(len(sql_n))
print(len(safe_n))

5000
5000
5000


In [16]:
dfs = []
  # Notice the double brackets!

df_list = [xss_n[['text', 'label']],  sql_n[['text', 'label']],safe_n[['text', 'label']]]

In [17]:
combined = pandas.concat(df_list,ignore_index=True)

In [18]:
combined.to_csv("csv_files/dt-v2.csv")